In [ ]:
# !pip uninstall -y foresight
#!pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight

In [ ]:
# !pip install langchain_openai
# !pip install langchain

In [2]:
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader 
from langchain_openai import ChatOpenAI


In [ ]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
# room_id = '09073d6d-6d79-4c98-bbea-4dd82106c098'
# board_id = '68993aeb-0ee6-4f95-b461-0f918fc7d969'
# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [ ]:
ps3 = PySage3(conf, prod_type)
cb = ps3.rooms['e3233abf-a47b-4c16-81ec-c6e985c67a5a'].boards['479d5114-a017-4b6e-973d-8933c46b8837']

In [8]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
#llm_4o.invoke("hello")

In [5]:
example_prompt = PromptLoader.create_prompt("src/prompts/", "few_shot", "reframe_select_apps")
example_prompt

FewShotPromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"updated_query": {"title": "Updated Query", "description": "The new reframed query str describing the items referenced in this query", "type": "string"}}, "required": ["updated_query"]}\n```'}, examples=[{'example_query': 'Can you summarize all the red stickies', 'example_updated_query': 'Select the red stickies'}, {'example_query': 'Can you delete all the pdf files', 'example_updated_query': 'Select the pdf files'}, {'example_que

In [14]:
q = 'Can you delete all the red stickies with a greeting or hello'
prompt_text = example_prompt.format(query=q)
print(prompt_text)

You are a meta-agent that needs to instruct a specialized agent to select the items to which a
query applies in a new updated_query. The new_updated query serves to identify the items which are first loaded.
A second agent then completes the desired action.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"updated_query": {"title": "Updated Query", "description": "The new reframed query str describing the items referenced in this query", "type": "string"}}, "required": ["updated_query"]}
```

Examples:
query: Can you summarize all the red stickies
updated_query: Select the r

In [15]:
(example_prompt | llm_4o).invoke({"query":q})

AIMessage(content='```json\n{\n  "updated_query": "Select the red stickies with a greeting or hello"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 302, 'total_tokens': 325}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-708ac5d4-efd0-418c-8307-bbe7d1b6725c-0')